## Movie_Success_Factor
###### 영화 및 TV프로그램 데이터베이스 기업인 tmdb의 영화 데이터를 활용했습니다
###### 예산 대비 수익이 높은 영화의 특징을 중심으로 분석해 보겠습니다

#### EDA

In [317]:
import warnings
warnings.filterwarnings('ignore')

In [242]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [243]:
movies = pd.read_csv('tmdb_5000_movies.csv')
movies.head(2)

,budget,genres,homepage,id,keywords,...,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",...,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",...,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [244]:
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [245]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [246]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


#### Preprocessing

In [247]:
og_movies = movies.copy()
og_credits = credits.copy()
movies = movies[['id', 'budget', 'genres', 'title', 'release_date', 'revenue', 'vote_average', 'vote_count']]
credits = credits[['movie_id', 'crew', 'cast']]

In [248]:
# movies와 credits 데이터를 한 dataframe에서 보기 위해 movies.id = credits.movie_id로 join
df = pd.merge(movies, credits, left_on='id', right_on='movie_id').drop('movie_id', axis=1)
df.head(2)

,id,budget,genres,title,release_date,revenue,vote_average,vote_count,crew,cast
0,19995,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Avatar,2009-12-10,2787965087,7.2,11800,"[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""..."
1,285,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",Pirates of the Caribbean: At World's End,2007-05-19,961000000,6.9,4500,"[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa..."


In [249]:
# 영화 수익률을 보기 위해 roi 컬럼을 추가
df['roi'] = df['revenue'] / df['budget']
df.head(2)

,id,budget,genres,title,release_date,...,vote_average,vote_count,crew,cast,roi
0,19995,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Avatar,2009-12-10,...,7.2,11800,"[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",11.763566
1,285,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",Pirates of the Caribbean: At World's End,2007-05-19,...,6.9,4500,"[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...",3.203333


In [250]:
# crew(제작진) 컬럼 내용 중 감독 정보만 보기 위한 작업
df['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [251]:
import ast

In [252]:
# ast.literal_eval 함수 사용 시 딕셔너리로 잘 변환되는지 확인
print(ast.literal_eval(df['crew'][0]))

[{'credit_id': '52fe48009251416c750aca23', 'department': 'Editing', 'gender': 0, 'id': 1721, 'job': 'Editor', 'name': 'Stephen E. Rivkin'}, {'credit_id': '539c47ecc3a36810e3001f87', 'department': 'Art', 'gender': 2, 'id': 496, 'job': 'Production Design', 'name': 'Rick Carter'}, {'credit_id': '54491c89c3a3680fb4001cf7', 'department': 'Sound', 'gender': 0, 'id': 900, 'job': 'Sound Designer', 'name': 'Christopher Boyes'}, {'credit_id': '54491cb70e0a267480001bd0', 'department': 'Sound', 'gender': 0, 'id': 900, 'job': 'Supervising Sound Editor', 'name': 'Christopher Boyes'}, {'credit_id': '539c4a4cc3a36810c9002101', 'department': 'Production', 'gender': 1, 'id': 1262, 'job': 'Casting', 'name': 'Mali Finn'}, {'credit_id': '5544ee3b925141499f0008fc', 'department': 'Sound', 'gender': 2, 'id': 1729, 'job': 'Original Music Composer', 'name': 'James Horner'}, {'credit_id': '52fe48009251416c750ac9c3', 'department': 'Directing', 'gender': 2, 'id': 2710, 'job': 'Director', 'name': 'James Cameron'}, 

In [253]:
df['crew'] = df['crew'].apply(ast.literal_eval)

In [254]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director': return i['name']

In [255]:
# 제작진 중 감독 정보만 확인하기 위해 director 컬럼 추가
df['director'] = df['crew'].apply(get_director)

In [256]:
df.head(2)

,id,budget,genres,title,release_date,...,vote_count,crew,cast,roi,director
0,19995,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Avatar,2009-12-10,...,11800,"[{'credit_id': '52fe48009251416c750aca23', 'de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",11.763566,James Cameron
1,285,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",Pirates of the Caribbean: At World's End,2007-05-19,...,4500,"[{'credit_id': '52fe4232c3a36847f800b579', 'de...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...",3.203333,Gore Verbinski


In [257]:
# 마찬가지로 출연진 컬럼의 내용 확인 
df['cast'][0]

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [258]:
# 감독과 달리 주연배우만 추출이 어려우니 배우들 이름을 확인하는 cast_name 컬럼 생성 
df['cast_name'] = df['cast'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)])
df.head(2)

,id,budget,genres,title,release_date,...,crew,cast,roi,director,cast_name
0,19995,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",Avatar,2009-12-10,...,"[{'credit_id': '52fe48009251416c750aca23', 'de...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",11.763566,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave..."
1,285,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",Pirates of the Caribbean: At World's End,2007-05-19,...,"[{'credit_id': '52fe4232c3a36847f800b579', 'de...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...",3.203333,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ..."


In [259]:
# 장르도 복수 개의 장르가 표기된 경우가 많으므로 하나의 장르만 뽑아봅시다 
df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [260]:
df['genres'] = df['genres'].apply(ast.literal_eval)

In [261]:
def get_genres(x):
    if len(x) > 0: return x[0]['name']

In [262]:
# 가장 앞의 장르를 메인 장르로 판단하고 main_genre 컬럼 생성 
df['main_genre'] = df['genres'].apply(get_genres)
df.head(2)

,id,budget,genres,title,release_date,...,cast,roi,director,cast_name,main_genre
0,19995,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Avatar,2009-12-10,...,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",11.763566,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",Action
1,285,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Pirates of the Caribbean: At World's End,2007-05-19,...,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...",3.203333,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Adventure


In [263]:
# 그 외에 형변환 작업 필요한 컬럼 확인 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   budget        4803 non-null   int64  
 2   genres        4803 non-null   object 
 3   title         4803 non-null   object 
 4   release_date  4802 non-null   object 
 5   revenue       4803 non-null   int64  
 6   vote_average  4803 non-null   float64
 7   vote_count    4803 non-null   int64  
 8   crew          4803 non-null   object 
 9   cast          4803 non-null   object 
 10  roi           3913 non-null   float64
 11  director      4773 non-null   object 
 12  cast_name     4803 non-null   object 
 13  main_genre    4775 non-null   object 
dtypes: float64(2), int64(4), object(8)
memory usage: 525.5+ KB


In [264]:
# release_date 컬럼의 데이터가 형태는 제대로 들어가 있지만(%Y-%m-%D) object 형태임을 확인
# 날짜형태로 형변환 
df['release_date'] = pd.to_datetime(df['release_date'])
df['id'] = df['id'].astype(str)

In [265]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            4803 non-null   object        
 1   budget        4803 non-null   int64         
 2   genres        4803 non-null   object        
 3   title         4803 non-null   object        
 4   release_date  4802 non-null   datetime64[ns]
 5   revenue       4803 non-null   int64         
 6   vote_average  4803 non-null   float64       
 7   vote_count    4803 non-null   int64         
 8   crew          4803 non-null   object        
 9   cast          4803 non-null   object        
 10  roi           3913 non-null   float64       
 11  director      4773 non-null   object        
 12  cast_name     4803 non-null   object        
 13  main_genre    4775 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(8)
memory usage: 525.5+ KB


In [266]:
# 나중에 분석이 편하도록 연도와 월에 해당하는 컬럼을 생성 
# 여기서 int64형으로 저장하려고 했으나 결측치 행이 있어 astype 함수 활용에 오류 발생 
df['year'] = df['release_date'].dt.year
df['month'] = df['release_date'].dt.month
df.head(2)

,id,budget,genres,title,release_date,...,director,cast_name,main_genre,year,month
0,19995,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Avatar,2009-12-10,...,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",Action,2009.0,12.0
1,285,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Pirates of the Caribbean: At World's End,2007-05-19,...,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Adventure,2007.0,5.0


In [267]:
# 확인해보니까 해당 컬럼은 하나고 내용도 분석에 큰 의미가 없어 보입니다 
df.loc[df['release_date'].isna(), :]

,id,budget,genres,title,release_date,...,director,cast_name,main_genre,year,month
4553,380097,0,[],America Is Still the Place,NaT,...,None,[],None,NaN,NaN


In [268]:
# 과감하게 제거 후
df = df.dropna(subset='release_date')
df.shape

(4802, 16)

In [269]:
# 나머지 컬럼들은 연도와 월을 정수로 변환 
df['year'] = df['year'].astype('int64')
df['month'] = df['month'].astype('int64')
df.head(2)

,id,budget,genres,title,release_date,...,director,cast_name,main_genre,year,month
0,19995,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",Avatar,2009-12-10,...,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",Action,2009,12
1,285,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Pirates of the Caribbean: At World's End,2007-05-19,...,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley, ...",Adventure,2007,5


In [270]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4802 entries, 0 to 4802
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            4802 non-null   object        
 1   budget        4802 non-null   int64         
 2   genres        4802 non-null   object        
 3   title         4802 non-null   object        
 4   release_date  4802 non-null   datetime64[ns]
 5   revenue       4802 non-null   int64         
 6   vote_average  4802 non-null   float64       
 7   vote_count    4802 non-null   int64         
 8   crew          4802 non-null   object        
 9   cast          4802 non-null   object        
 10  roi           3913 non-null   float64       
 11  director      4773 non-null   object        
 12  cast_name     4802 non-null   object        
 13  main_genre    4775 non-null   object        
 14  year          4802 non-null   int64         
 15  month         4802 non-null   int64        

In [271]:
df.isna().sum() / df.shape[0]

id              0.000000
budget          0.000000
genres          0.000000
title           0.000000
release_date    0.000000
                  ...   
director        0.006039
cast_name       0.000000
main_genre      0.005623
year            0.000000
month           0.000000
Length: 16, dtype: float64

In [272]:
# release_date 컬럼 결측치 제거 후에도 roi, main_genre 컬럼에 결측치가 존재함을 확인
# 메인 장르는 결측치 비율이 매우 낮고 roi가 결측치인 경우 흥행 요인 분석 자체가 어렵습니다 
# 따라서 이번 분석에서는 모든 결측치를 단순 제거하기로 결정
df.dropna(inplace=True)

In [273]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3905 entries, 0 to 4799
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            3905 non-null   object        
 1   budget        3905 non-null   int64         
 2   genres        3905 non-null   object        
 3   title         3905 non-null   object        
 4   release_date  3905 non-null   datetime64[ns]
 5   revenue       3905 non-null   int64         
 6   vote_average  3905 non-null   float64       
 7   vote_count    3905 non-null   int64         
 8   crew          3905 non-null   object        
 9   cast          3905 non-null   object        
 10  roi           3905 non-null   float64       
 11  director      3905 non-null   object        
 12  cast_name     3905 non-null   object        
 13  main_genre    3905 non-null   object        
 14  year          3905 non-null   int64         
 15  month         3905 non-null   int64        

#### 분석 1: 연도별 흥행 수익 확인하기

In [274]:
# 시각화를 활용해 연도별 흥행 수익의 전체적인 추세를 보겠습니다 
import plotly.express as px

In [275]:
revenue_by_year = df.groupby('year')[['revenue']].sum().reset_index()

In [276]:
fig = px.line(data_frame=revenue_by_year, x='year', y='revenue', title='연도별 영화 흥행 수익')
fig.show()

###### 영화 흥행 수익의 빠른 발전을 확인할 수 있고, 2010년대 들어 급격한 상승도 눈에 띕니다. 

In [277]:
df['year'].describe()

count    3905.000000
mean     2002.206146
std        12.750332
min      1916.000000
25%      1999.000000
50%      2005.000000
75%      2011.000000
max      2016.000000
Name: year, dtype: float64

In [278]:
df['release_date'].max()

Timestamp('2016-09-16 00:00:00')

###### 2015 -> 2016년 사이의 급격한 하락세는 해당 데이터셋이 2016년 9월 개봉작까지의 데이터이고 
###### 따라서 아직 벌어들일 수익이 남은 2016년 영화들의 roi가 확정되지 않았기 때문으로 보입니다. 

#### 분석 2: 가장 흥행한 10개의 영화들

In [279]:
# 가장 높은 수익을 낸 10개 영화의 수익을 시각화해 보겠습니다.
top10 = df.groupby('title')[['revenue', 'year']].sum().reset_index().sort_values(by='revenue', ascending=False).head(10)
fig = px.bar(data_frame=top10, x='title', y='revenue', title='최고 흥행수익 영화')
fig.show()

In [280]:
top10

,title,revenue,year
310,Avatar,2787965087,2009
3591,Titanic,1845034188,1997
2806,The Avengers,1519557910,2012
1569,Jurassic World,1513528810,2015
1136,Furious 7,1506249360,2015
311,Avengers: Age of Ultron,1405403694,2015
1127,Frozen,1274219009,2013
1500,Iron Man 3,1215439994,2013
1847,Minions,1156730962,2015
553,Captain America: Civil War,1153304495,2016


###### 마블 영화들이 다수 눈에 띄는 가운데 압도적인 수익의 아바타, 1997년작 타이타닉의 선전도 인상적입니다. 

In [281]:
# 최고 수익률 영화들도 살펴보려 했으나 예산 데이터가 0이라 수익률이 무한대로 산출된 영화들이 다수 발견됩니다 
# 수치로만 살펴본 결과 극단적인 수익률의 경우 큰 의미가 없는 것 같습니다.
top10_roi = df.loc[df['budget'] != 0, :].groupby('title')['roi'].sum().reset_index().sort_values(by='roi', ascending=False).head(10)
# fig = px.bar(data_frame=top10_roi, x='title', y='roi', title='최고 수익률 영화')
# fig.show()
top10_roi

,title,roi
1791,Modern Times,8.500000e+06
1930,Nurse 3-D,1.000000e+06
2009,Paranormal Activity,1.289039e+04
2632,Tarnation,5.330339e+03
2730,The Blair Witch Project,4.133333e+03
915,Eraserhead,7.000000e+02
2050,Pink Flamingos,5.000000e+02
2585,Super Size Me,4.396166e+02
2896,The Gallows,4.266441e+02
1974,Open Water,4.205227e+02


In [282]:
# 예산, 투표수 부문 최고 영화들을 살펴보겠습니다. 
# 투표수는 TMDB 사이트의 리뷰 수에 해당하는 것으로 보이며, 흥행 수준 판단단에 유의미한 지표로 해석할 수 있습니다. 
top10_budget = df.groupby('title')[['budget']].sum().reset_index().sort_values(by='budget', ascending=False).head(10)
fig = px.bar(data_frame=top10_budget, x='title', y='budget', title='최고 제작비 영화')
fig.show()

In [283]:
top10_vote = df.groupby('title')[['vote_count']].sum().reset_index().sort_values(by='vote_count', ascending=False).head(10)
fig = px.bar(data_frame=top10_vote, x='title', y='vote_count', title='최고 리뷰수 영화', labels=dict(title='title', vote_count='review'))
fig.show()

###### 예산이 높을수록, 리뷰가 많을수록 흥행 수익도 높은 것으로 보이나 뚜렷한 관계가 존재한다고 단정짓기는 어렵습니다. 
###### 이후에 상관관계 분석을 진행하면 더 자세히 알아볼 수 있을 것 같습니다. 

#### 분석 3: 흥행에 가장 성공한 감독과 배우

In [284]:
# 최고 흥행 수익을 기록한 감독 10명의 수익을 시각화해 보겠습니다. 
top10_director = df.groupby('director')[['revenue']].sum().reset_index().sort_values(by='revenue', ascending=False).head(10)
fig = px.bar(data_frame=top10_director, x='director', y='revenue', title='최고 흥행수익 감독')
fig.show()

In [285]:
# 10명의 영화 감독이 전체 영화 흥행 수익의 약 12%에 해당하는 성과를 거두었습니다.
# 전체 영화감독 중 10명은 약 0.6%에 해당하는 수치입니다. 
top10_director['ratio'] = top10_director['revenue'] / df['revenue'].sum()
top10_director

,director,revenue,ratio
1605,Steven Spielberg,9147393164,0.023152
1317,Peter Jackson,6498642820,0.016448
680,James Cameron,5883569439,0.014892
1122,Michael Bay,5832524638,0.014762
274,Christopher Nolan,4227483234,0.010700
242,Chris Columbus,3725631503,0.009430
1444,Robert Zemeckis,3590622002,0.009088
567,George Lucas,3339113893,0.008451
1657,Tim Burton,3337418241,0.008447
1402,Ridley Scott,3189557997,0.008073


In [286]:
# 같은 방식으로 최고 흥행 수익을 기록한 배우의 수익도 보겠습니다.
# 현재 df의 cast_name 컬럼이이 배우들의 명단 형태로 저장되어 있으므로 explode를 통해 풀어줍니다. 
temp_cast = df[['revenue', 'cast_name']].explode('cast_name')
temp_cast.head(2)

,revenue,cast_name
0,2787965087,Sam Worthington
0,2787965087,Zoe Saldana


In [287]:
top10_cast = temp_cast.groupby('cast_name')[['revenue']].sum().reset_index().sort_values('revenue', ascending=False).head(10)
fig = px.bar(data_frame=top10_cast, x='cast_name', y='revenue', title='최고 흥행수익 배우')
fig.show()

In [288]:
# 영화 한 편에는 감독이 한 명이지만, 여러 명의 배우가 출연합니다. 
# 따라서 전체 영화 흥행수익 대비 배우의 비중을 정량적 지표로 보기에는 무리가 있습니다. 
# 영화 산업에서 배우의 상대적인 영향력을 확인할 수는 있을 것 같습니다. 
top10_cast['ratio'] = top10_cast['revenue'] / df['revenue'].sum()

In [289]:
# 또한 많은 작품에 출연한 배우가 이 분석에서 유리하기 때문에 출연작 당 평균 수입도 살펴볼 수 있습니다. 
# 출연작 당 평균 수입이 높은 배우일수록 이른바 '흥행 보증 수표'에 해당합니다. 
from collections import Counter
top10_cast['films'] = pd.Series()
for cast in top10_cast['cast_name']:
    top10_cast.loc[top10_cast['cast_name'] == cast, 'films'] = Counter(temp_cast['cast_name'])[cast]
top10_cast['revenue_per_film'] = (top10_cast['revenue'] / top10_cast['films']).astype('int64')
top10_cast.sort_values(by='revenue_per_film', ascending=False)

,cast_name,revenue,ratio,films,revenue_per_film
43931,Stan Lee,17364063582,0.043949,27,643113466
21875,Jess Harnell,9633458775,0.024383,16,602091173
18754,Ian McKellen,9710670395,0.024578,18,539481688
18636,Hugo Weaving,10822190781,0.027391,21,515342418
23547,John Ratzenberger,11038044745,0.027938,22,501729306
15620,Frank Welker,11614837160,0.029398,33,351964762
7235,Cate Blanchett,9726416776,0.024618,29,335393681
46582,Tom Cruise,8993387534,0.022763,33,272526894
41822,Samuel L. Jackson,14806065788,0.037475,58,255276996
34423,Morgan Freeman,9275477679,0.023477,46,201640819


###### 일부 유명한 감독과 배우들이 차지하는 흥행 수익의 비중이 높습니다. 
###### 해당 데이터셋이 100년 간 영화들의 데이터에 해당하므로 특정 감독이나 배우의 영향력을 판단하기 위해서는
###### 해당 인물의 활동 기간에 개봉한 영화들을 추출해서 살펴볼 수 있습니다. 

#### 분석 4: 장르와 흥행 수익의 관계

In [290]:
# 영화의 장르가 흥행 수익과 관련이 있다고 생각되므로 흥행 수익이 좋은 장르를 확인해 보겠습니다.
fig = px.box(data_frame=df, x='revenue', y='main_genre', hover_name='title')
fig.show()

###### 액션, 드라마 등 장르의 히트작이 눈에 띄지만 중앙값이나 Q1~Q3구간이 특별이 높아 보이지는 않습니다. 

In [291]:
# 장르별 흥행 수익의 평균과 합계를를 확인해 보겠습니다. 
genre_avg_revenue = df.groupby('main_genre')[['revenue']].mean().reset_index()
genre_sum_revenue = df.groupby('main_genre')[['revenue']].sum().reset_index()

In [292]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2, subplot_titles=['장르별 흥행 수익 평균', '장르별 흥행 수익 합계'])
fig.add_trace(go.Bar(x=genre_avg_revenue['main_genre'], y=genre_avg_revenue['revenue'], name='avg'), row=1, col=1)
fig.add_trace(go.Bar(x=genre_sum_revenue['main_genre'], y=genre_sum_revenue['revenue'], name='sum'), row=1, col=2)
fig.show()

###### 장르별 수익 평균은 애니메이션 > 어드벤처 > 가족 > SF > 판타지 > 액션 순
###### 장르별 수익 합계는 액션 > 어드벤처 > 드라마 > 코미디 > 애니메이션 순으로 나타납니다. 
###### 한편 시대와 계절에 따라 유행하는 장르가 있고, 이것이 흥행 수익과 연관이 있을 것이라는 가정을 해볼 수 있습니다. 

In [314]:
# 연도에 따른 장르별 수익을 비교하기 위해 간단한 시각화를 진행해 봅시다. 
revenue_by_year_genre = df.query('year >= 1990').groupby(['year', 'main_genre'])[['revenue']].sum().reset_index()
fig = px.bar(data_frame=revenue_by_year_genre, x='year', y='revenue', color='main_genre')
fig.show()

In [326]:
# 연도별로 각 장르가 해당 연도 전체 영화 흥행수익에서 차지하는 비율(%)을 확인해 보겠습니다. 
revenue_by_year_genre_pct = pd.pivot_table(data=df.query('year>=1990'), index='year', columns='main_genre', values='revenue', aggfunc=sum, fill_value=0, margins=True)
revenue_by_year_genre_pct = 100 * revenue_by_year_genre_pct.div(revenue_by_year_genre_pct.iloc[:,-1], axis=0).drop('All').drop('All', axis=1)
revenue_by_year_genre_pct = pd.melt(revenue_by_year_genre_pct.reset_index(), id_vars='year', value_name='pct')
revenue_by_year_genre_pct.sort_values(by='year')

,year,main_genre,pct
0,1990,Action,19.916921
432,1990,TV Movie,0.000000
405,1990,Science Fiction,1.478356
378,1990,Romance,11.983109
54,1990,Animation,0.000000
...,...,...,...
404,2016,Romance,0.874497
431,2016,Science Fiction,3.761350
458,2016,TV Movie,0.000000
242,2016,Fantasy,4.810644


In [327]:
fig = px.bar(data_frame=revenue_by_year_genre_pct, x='year', y='pct', color='main_genre')
fig.show()

###### 2010년대 이후 액션 장르의 선전과 1990~2000년대 드라마 장르의 선전 등이 눈에 띕니다. 

In [329]:
# 월별로 유행한 장르에 대해서도 살펴보겠습니다.
revenue_by_month_genre = df.query('year>=1990').groupby(['month', 'main_genre'])[['revenue']].sum().reset_index()
fig = px.bar(data_frame=revenue_by_month_genre, x='month', y='revenue', color='main_genre')
fig.show()

###### 여름과 연말 시즌을 앞두고 개봉한 영화들의 흥행 수익 합이 높음을 확인할 수 있습니다.

In [331]:
# 계절별로 유행한 장르의 비율만을 조금 더 자세히 살펴보겠습니다.
revenue_by_month_genre_pct = pd.pivot_table(data=df.query('year>=1990'), index='month', columns='main_genre', values='revenue', aggfunc=sum, fill_value=0, margins=True)
revenue_by_month_genre_pct = 100 * revenue_by_month_genre_pct.div(revenue_by_month_genre_pct.iloc[:,-1], axis=0).drop('All').drop('All', axis=1)
revenue_by_month_genre_pct = pd.melt(revenue_by_month_genre_pct.reset_index(), id_vars='month', value_name='pct')
revenue_by_month_genre_pct

,month,main_genre,pct
0,1,Action,30.795424
1,2,Action,21.360752
2,3,Action,25.562012
3,4,Action,36.264242
4,5,Action,18.635487
...,...,...,...
235,8,Western,1.042351
236,9,Western,0.316362
237,10,Western,0.000000
238,11,Western,0.000000


In [333]:
fig = px.bar(data_frame=revenue_by_month_genre_pct, x='month', y='pct', color='main_genre')
fig.show()

###### 액션/어드벤처 장르는 대체로 봄~여름에 흥행 수익이 높았습니다. 
###### 드라마는 가을~겨울, 코미디는 겨울에 비교적 그 수익이 높았습니다. 

#### 분석 5: 예산/투표수/평점과 흥행 수익의 관계

In [293]:
# 감독과 배우, 장르 외에도 예산, 리뷰수, 평점 등 다양한 요소들이 흥행 수익에 영향을 줄 수 있습니다.
# 흥행 수익과 해당 요인들 간 상관관계를 확인해 보겠습니다. 
df[['budget', 'revenue', 'vote_average', 'vote_count']].corr()

,budget,revenue,vote_average,vote_count
budget,1.000000,0.710727,0.013522,0.560056
revenue,0.710727,1.000000,0.196757,0.768594
vote_average,0.013522,0.196757,1.000000,0.354340
vote_count,0.560056,0.768594,0.354340,1.000000


In [294]:
fig = px.imshow(df[['budget', 'revenue', 'vote_average', 'vote_count']].corr(), text_auto='.2f', color_continuous_scale='Teal')
fig.show()

###### 수익(revevue)는 예산/제작비(budget), 투표/리뷰수(vote_count)등과 0.7 이상의 유의미한 상관관계를 가집니다. 
###### 투표/리뷰 점수(vote_average)는 예상 외로 수익과 큰 상관관계를 보이지는 않는 모습입니다. 

In [307]:
for x in ['budget', 'vote_count', 'vote_average']:
    fig = px.scatter(data_frame=df, x=x, y='revenue', hover_name='title', size='revenue', color='revenue',
                     color_continuous_scale='Burg', width=700, height=600, trendline='ols')
    fig.show()

In [303]:
# 이번엔 흥행 성적이 좋은 100개의 영화들로 범위를 한정하여 확인해 보겠습니다.
top100_revenue = df.sort_values('revenue', ascending=False).head(100)
fig = px.imshow(top100_revenue[['budget', 'revenue', 'vote_average', 'vote_count']].corr(), text_auto='.2f', color_continuous_scale='Purp')
fig.show()

In [306]:
for x in ['budget', 'vote_count', 'vote_average']:
    fig = px.scatter(data_frame=top100_revenue, x=x, y='revenue', hover_name='title', size='revenue', color='revenue',
                     color_continuous_scale='Emrld', width=700, height=600, trendline='ols', trendline_color_override='green')
    fig.show()

###### 상관관계 분석 결과 예산과 투표수는 흥행 수익과의 높은 양의 상관관계를 보입니다. 
###### 평점 평균은 비교적 낮은 양의 상관관계를 보였습니다. 
###### 흥행 성적이 좋은 100개의 영화로 범위를 한정했을 때 이 상관관계들은 더 낮아짐을 확인할 수 있었습니다. 

#### 분석 6: ROI 관점에서 흥행이 성공한 영화의 특징

###### ROI(예산 대비 수익)의 관점에서 흥행에 성공한 영화들의 특징을 간략하게 살펴보겠습니다.

In [336]:
# 흥행 수익이 높은 300개 영화들의 ROI와 수익을 확인해 보겠습니다.
top300_revenue = df.sort_values('revenue', ascending=False).head(300)

In [335]:
fig = px.scatter(data_frame=top300_revenue, x='roi', y='revenue', hover_name='title', size='revenue', color='main_genre', width=700, height=600)
fig.show()

In [337]:
fig = px.box(data_frame=top300_revenue, x='roi', y='main_genre', hover_name='title')
fig.show()

###### 드라마와 로맨스 장르 영화들이 대체로 높은 ROI를 기록했습니다. 
###### 분석 전반에 걸쳐 액션 장르의 흥행작들을 다수 살펴보았지만 예산이 큰 경우가 많은 장르의 특성상 ROI는 높지 않았습니다. 